In [196]:
import torch
import random
import numpy as np
import re

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_female"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool
FAST_ALIGN="/home/vzhekova/fast_align/build/fast_align" # path to the fast_align tool
TERCOM = "/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE"

In [197]:
# check if we can connect to the GPU with PyTorch
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: GeForce GTX 1080 Ti


In [198]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_female


In [20]:
# Extract sentences
!cut -f3 -d'	' en.txt > en_sentences.txt

In [21]:
# Extract sentences containing 'because' and remove the second part of the clause
# 330 unique sentences in total
with open('en_sentences.txt', 'r') as fin, open('en_original.txt', 'w') as fout:
    for line in fin:
        sentence = ''
        tokens = line.split(" ")
        for token in tokens:
            if token == 'because':
                print(sentence + '.', end='\n', file=fout)
            sentence = sentence + token.replace(',', '') + ' '

In [23]:
# Modify gender ambiguous words with gender

# List with source words
words = set() # set forces uniqueness
with open('words.txt', 'r') as fin:
    for line in fin:
        words.add(line.strip())
        
with open('en_original.txt') as in_file, open('en_disambiguated.txt', 'w') as out_file: 
    for line in in_file:
        sentence = line.split(' ')
        for token in sentence:
            if (token.replace(',', '') in words): # tokens often contain ","
                token_pos = sentence.index(token)
                sentence[token_pos] = "male " + token # could also replace with "female"
        print(' '.join(sentence), end='', file=out_file)

# Translation English-German

In [24]:
# Tokenization
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mpn = MosesPunctNormalizer()
mt_en = MosesTokenizer(lang='en')
md_en = MosesDetokenizer(lang='en')

with open('en_original.txt') as fin, open('tok.en_original.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout) 
        
with open('en_disambiguated.txt') as fin, open('tok.en_disambiguated.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished tokenizing.')

Finished tokenizing.


In [25]:
# Dividing text into subword units

!$FASTBPE applybpe bpe.en_original.en tok.en_original.en bpecodes.en
!$FASTBPE applybpe bpe.en_disambiguated.en tok.en_disambiguated.en bpecodes.en

print('Finished subword.')

Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from tok.en_original.en ...
Read 2733 words (470 unique) from text file.
Applying BPE to tok.en_original.en ...
Modified 2733 words from text file.
Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from tok.en_disambiguated.en ...
Read 3388 words (471 unique) from text file.
Applying BPE to tok.en_disambiguated.en ...
Modified 3388 words from text file.
Finished subword.


In [26]:
# Binarize text
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.en_original \
    --only-source \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_original_en-de \
    --workers 8

2023-05-09 11:28:37 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcd

In [27]:
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.en_disambiguated \
    --only-source \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_disambiguated_en-de \
    --workers 8

2023-05-09 11:29:02 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_disambiguated_en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en',

In [209]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
NBEST = 100
BEAM = 100

In [200]:
# Generate N hypothesis
!fairseq-generate data-bin_original_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > original_en-de.decode_Beam_100.log

2023-07-18 15:19:46 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [210]:
# Generate N hypothesis
!fairseq-generate data-bin_disambiguated_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_en-de.decode_Beam_100.log

2023-07-18 19:54:42 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

# Backtranslation German-English

In [203]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_en-de.decode_Beam_100.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_100.txt
!grep ^H disambiguated_en-de.decode_Beam_100.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_100.txt

In [204]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp_original_100.de hyp_original_100.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_disambiguated_100.de hyp_disambiguated_100.txt bpecodes.de

print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original_100.txt ...
Read 277261 words (3716 unique) from text file.
Applying BPE to hyp_original_100.txt ...
Modified 277261 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_disambiguated_100.txt ...
Read 281702 words (3780 unique) from text file.
Applying BPE to hyp_disambiguated_100.txt ...
Modified 281702 words from text file.
Finished subword.


In [205]:
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original_100 \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_de-en_100 \
    --workers 8

2023-07-18 15:45:33 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_de-en_100', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', 

In [206]:
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_disambiguated_100 \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_disambiguated_de-en_100 \
    --workers 8

2023-07-18 15:45:39 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_disambiguated_de-en_100', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='

In [211]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"
NBEST = 100
BEAM = 100

In [208]:
# Generate N hypothesis
!fairseq-generate data-bin_original_de-en_100  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > original_de-en.decode_Beam_100_backtranslation.log

2023-07-18 15:46:37 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [212]:
# Generate N hypothesis
!fairseq-generate data-bin_disambiguated_de-en_100  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 4 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_de-en.decode_Beam_100_backtranslation.log

2023-07-18 20:00:15 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [213]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_de-en.decode_Beam_100_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_back_100.txt
!grep ^H disambiguated_de-en.decode_Beam_100_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_back_100.txt

In [214]:
# Detokenize text        
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

md_en = MosesDetokenizer(lang='en')

with open('hyp_original_back_100.txt', encoding='utf8') as fin, open('original_back_100.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_disambiguated_back_100.txt', encoding='utf8') as fin, open('disambiguated_back_100.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


# Statistics on translations

In [215]:
# List with original source sentences
source = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
# List with disambiguated source sentences
source_disambiguated = []
with open('en_disambiguated.txt', 'r') as fin:
    for line in fin:
        source_disambiguated.append(line.strip())
    
# List with nbest sentences for every source in original
nbest_original = []
counter = 0
temp = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated            
nbest_disambiguated = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated.append(temp)
            counter = 0
            temp = []
            
print(len(source))
print(len(nbest_original))
print(len(nbest_disambiguated))

330
330
330


## Count unique sentences

In [216]:
# Count unique sentences in source nbest list for each source sentence of original; 9.945454545454545
# Value should be 10, because beam search generates 10 unique sentences
unique_sent = []
for source_nbest in nbest_original:
    num_values = len(set(source_nbest))
    #print(num_values)
    unique_sent.append(num_values)
    
#print(unique_sent)
print(sum(unique_sent)/330) # average

98.7090909090909


In [217]:
# Count unique sentences in source nbest list for each source sentence of modified; 9.954545454545455
unique_sent = []
for source_nbest in nbest_disambiguated:
    num_values = len(set(source_nbest))
    #print(num_values)
    unique_sent.append(num_values)
    
#print(unique_sent)
print(sum(unique_sent)/330) # average

98.0060606060606


## Count unique words

In [ ]:
# Count unique words in source nbest list for each source sentence of original; 16.836363636363636
import spacy

sp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS


unique_words = []
counter = 0
for source_nbest in nbest_original:
    words = set()
    for sent in source_nbest:
        tokens = sp(sent)
        for token in tokens:
            if token.text not in stopwords:    # checking whether the word is a stop word
                words.add(token.text)
    num_values = len(words)
    unique_words.append(num_values)
    
    counter += 1
    print(counter)
    
#print(unique_words)
print(sum(unique_words)/330)

In [ ]:
# Count unique words in source nbest list for each source sentence of modified; 17.64848484848485
# !!! This is normal to generate more unique words, because the disambiguated sentences have more words in total
import spacy

sp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS


unique_words = []
counter = 0
for source_nbest in nbest_disambiguated:
    words = set()
    for sent in source_nbest:
        tokens = sp(sent)
        for token in tokens:
            if token.text not in stopwords:    # checking whether the word is a stop word
                words.add(token.text)
    num_values = len(words)
    unique_words.append(num_values)
    
    counter += 1
    print(counter)
    
#print(unique_words)
print(sum(unique_words)/330)

# Statistics on backtranslations

In [222]:
# List with original source sentences
source_original = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source_original.append(line.strip())
        
# List with disambiguated source sentences
source_disambiguated = []
with open('en_disambiguated.txt', 'r') as fin:
    for line in fin:
        source_disambiguated.append(line.strip())
    
# List with nbest sentences for every source in original 
nbest_original = []
counter = 0
temp = []
with open('original_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated
nbest_disambiguated = []
with open('disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_disambiguated.append(temp)
            counter = 0
            temp = []
            
print(len(nbest_original))
print(len(nbest_disambiguated))

330
330


## Source sentence reoccurrence

In [223]:
# Count how many times the source sentence occurs in the nbest list of original; 258
results = []
counter = 0
for sent in source_original:
    matches = 0
    for target in nbest_original[counter]: 
        if (sent == target):
            matches += 1
    results.append(matches)  
    counter += 1
    
print(sum(results)/330)
print(sum(x > 0 for x in results))

70.53333333333333
310


In [224]:
# Count how many times the source sentence occurs in the nbest list of disambiguated; 230
results = []
counter = 0
for sent in source_disambiguated:
    matches = 0
    for target in nbest_disambiguated[counter]: 
        if (sent == target):
            matches += 1
    results.append(matches)  
    counter += 1
    
print(sum(results)/330)
print(sum(x > 0 for x in results))

13.163636363636364
184


## Ambiguous source words reoccurrence


In [225]:
# List with source words
words = set() # set forces uniqueness
with open('words.txt', 'r') as fin:
    for line in fin:
        words.add(line.strip())

# Extract ambiguous words from source sentences
ambiguous_words = [] 
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break
        
print(ambiguous_words)
print(len(ambiguous_words))
        
# List with nbest sentences for every source
nbest_original = []
counter = 0
temp = []
with open('original_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_original.append(temp)
            counter = 0
            temp = []

nbest_disambiguated = []
with open('disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_disambiguated.append(temp)
            counter = 0
            temp = []  

print(len(nbest_original))
print(len(nbest_disambiguated))     

['developer', 'mechanic', 'mover', 'assistant', 'chief', 'salesperson', 'lawyer', 'cook', 'mover', 'farmer', 'CEO', 'hairdresser', 'developer', 'driver', 'auditor', 'CEO', 'guard', 'assistant', 'assistant', 'auditor', 'salesperson', 'manager', 'physician', 'laborer', 'physician', 'hairdresser', 'developer', 'farmer', 'receptionist', 'manager', 'cleaner', 'mechanic', 'writer', 'worker', 'editor', 'analyst', 'carpenter', 'cook', 'carpenter', 'cleaner', 'laborer', 'mechanic', 'mechanic', 'cook', 'farmer', 'CEO', 'librarian', 'chief', 'developer', 'nurse', 'lawyer', 'developer', 'mover', 'mover', 'worker', 'secretary', 'CEO', 'carpenter', 'sheriff', 'mechanic', 'analyst', 'assistant', 'chief', 'janitor', 'manager', 'supervisor', 'chief', 'worker', 'salesperson', 'lawyer', 'developer', 'sheriff', 'janitor', 'laborer', 'driver', 'mover', 'developer', 'janitor', 'salesperson', 'chief', 'laborer', 'guard', 'nurse', 'worker', 'laborer', 'lawyer', 'CEO', 'laborer', 'laborer', 'nurse', 'manager',

In [226]:
# Count how many times the source words occurs in the nbest list of original
results = []
counter = 0
for word in ambiguous_words:
    matches = 0
    for target in nbest_original[counter]: 
        if (word in target.split(" ")):
            matches += 1
    results.append(matches)  
    counter += 1
    
print(results)
print(sum(results)/330)
print(sum(x > 0 for x in results))

[5754, 9274, 1157, 6472, 687, 297, 4629, 7914, 4147, 7704, 1648, 4469, 3891, 9307, 4415, 4144, 5480, 6244, 6885, 4117, 781, 4571, 425, 5, 313, 3668, 7336, 6224, 6585, 7174, 4606, 8534, 3760, 6492, 6158, 8681, 6224, 7980, 6937, 6243, 45, 8425, 7240, 6431, 7917, 1958, 9635, 1267, 7354, 7277, 6396, 4080, 246, 953, 6324, 8810, 2097, 7655, 9023, 8321, 8888, 4752, 2331, 3312, 7746, 2278, 1378, 3770, 754, 5904, 7815, 9161, 2477, 53, 9594, 5643, 3788, 3886, 444, 1308, 252, 5038, 6127, 5720, 69, 6898, 893, 169, 202, 5801, 8957, 7345, 3235, 9515, 9999, 9327, 9472, 7716, 40, 7410, 4621, 6812, 9442, 3483, 425, 9543, 7882, 4848, 5778, 9880, 3923, 4189, 3219, 1729, 427, 9319, 6572, 3148, 1673, 776, 6685, 7577, 2528, 1869, 193, 9250, 8603, 8233, 7823, 9160, 6485, 8978, 8924, 8480, 4215, 1225, 8402, 8907, 922, 3573, 4243, 4543, 7195, 5955, 9597, 9964, 294, 845, 8920, 6952, 5207, 530, 9294, 446, 1376, 703, 7880, 511, 3106, 928, 5219, 145, 6053, 7529, 3423, 5417, 1397, 7649, 5588, 6328, 1018, 6595, 6382

In [227]:
# Count how many times the source words occurs in the nbest list of disambiguated
results = []
counter = 0
for word in ambiguous_words:
    matches = 0
    for target in nbest_disambiguated[counter]: 
        if (word in target.split(" ")):
            matches += 1
    results.append(matches)  
    counter += 1
    
print(results)
print(sum(results)/330)
print(sum(x > 0 for x in results))

[4488, 9337, 145, 8322, 524, 401, 4177, 8910, 2478, 3798, 778, 5041, 3286, 6942, 3088, 2325, 6125, 5664, 6328, 2586, 402, 4754, 356, 2, 322, 4485, 4527, 2575, 7164, 6352, 5223, 8708, 3369, 6842, 6325, 7855, 5658, 8753, 5585, 6070, 16, 8351, 6934, 8081, 4689, 1180, 9614, 898, 6202, 8039, 5696, 2725, 22, 0, 6759, 8994, 1011, 6470, 6094, 8278, 8209, 5734, 1527, 1936, 7414, 2079, 519, 4528, 585, 5639, 6425, 7583, 1227, 13, 8935, 72, 4350, 1790, 325, 831, 177, 5773, 6540, 7012, 8, 6460, 338, 96, 133, 6395, 7955, 5646, 5201, 8229, 9876, 4515, 9360, 7112, 5, 5528, 4972, 7676, 6175, 2569, 278, 8267, 7433, 4551, 5586, 9322, 2994, 4273, 2449, 423, 290, 7429, 3148, 3314, 723, 789, 5483, 6360, 52, 465, 127, 4770, 5000, 7523, 7285, 9104, 5574, 5492, 8362, 5290, 6810, 29, 7264, 7067, 791, 1343, 3146, 6366, 6419, 5776, 8604, 9208, 198, 768, 5091, 7503, 5262, 565, 9483, 0, 1005, 761, 4625, 442, 1608, 740, 5020, 93, 7981, 6868, 3370, 5016, 1059, 5281, 5322, 5874, 7, 3832, 3413, 6625, 56, 4553, 8843, 71

## Count unique sentences

In [228]:
# Count unique sentences in source nbest list for each source sentence of original; 46.06060606060606
unique_sent = []
for source_nbest in nbest_original:
    num_values = len(set(source_nbest))
    #print(num_values)
    unique_sent.append(num_values)
    
#print(unique_sent)
print(sum(unique_sent)/330) # average

2944.051515151515


In [229]:
# Count unique sentences in source nbest list for each source sentence of modified; 51.77272727272727
unique_sent = []
for source_nbest in nbest_disambiguated:
    num_values = len(set(source_nbest))
    #print(num_values)
    unique_sent.append(num_values)
    
#print(unique_sent)
print(sum(unique_sent)/330) # average

3162.3121212121214


## Count unique words

In [ ]:
# Count unique words in source nbest list for each source sentence of original; 22.593939393939394
import spacy

sp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS


unique_words = []
counter = 0
for source_nbest in nbest_original:
    words = set()
    for sent in source_nbest:
        tokens = sp(sent)
        for token in tokens:
            if token.text not in stopwords:    # checking whether the word is a stop word
                words.add(token.text)
    num_values = len(words)
    unique_words.append(num_values)
    
    counter += 1
    print(counter)
    
#print(unique_words)
print(sum(unique_words)/330)

In [ ]:
# Count unique words in source nbest list for each source sentence of modified; 22.348484848484848
import spacy

sp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS


unique_words = []
counter = 0
for source_nbest in nbest_disambiguated:
    words = set()
    for sent in source_nbest:
        tokens = sp(sent)
        for token in tokens:
            if token.text not in stopwords:    # checking whether the word is a stop word
                words.add(token.text)
    num_values = len(words)
    unique_words.append(num_values)
    
    counter += 1
    print(counter)
    
#print(unique_words)
print(sum(unique_words)/330)

# Word alignement (source-translation)

- Count how many unique ambiguous words are in total in source text
- Extract the position of the first ambiguous word from each sentence

In [230]:
# List with source words
words = set() # set forces uniqueness
with open('words.txt', 'r') as fin:
    for line in fin:
        words.add(line.strip())
        
ambiguous_words = set() # set forces uniqueness
positions_ambiguous_words = []

with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.add(token)
                position = tokens.index(token)
                positions_ambiguous_words.append(position)
                break
        
print(ambiguous_words)
print(len(ambiguous_words))
print(positions_ambiguous_words)
print(len(positions_ambiguous_words))

{'editor', 'assistant', 'patient', 'tailor', 'mechanic', 'firefighter', 'mover', 'laborer', 'developer', 'salesperson', 'janitor', 'cleaner', 'teenager', 'therapist', 'CEO', 'manager', 'clerk', 'chief', 'bartender', 'secretary', 'undergraduate', 'housekeeper', 'driver', 'examiner', 'specialist', 'programmer', 'dietitian', 'accountant', 'guard', 'advisor', 'broker', 'receptionist', 'sheriff', 'farmer', 'customer', 'baker', 'scientist', 'auditor', 'carpenter', 'supervisor', 'painter', 'librarian', 'writer', 'cook', 'nurse', 'attendant', 'physician', 'counselor', 'analyst', 'practitioner', 'lawyer', 'hairdresser', 'worker'}
53
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [231]:
# List with original source sentences
source = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
         
# List with nbest sentences for every source in original 
nbest_original = []
counter = 0
temp = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original.append(temp)
            counter = 0
            temp = []

print(len(source))
print(len(nbest_original))           
        
count = 0
with open('original_100_source-target_en-de.txt', 'w') as fout:
    while count < 330:
        for hyp in nbest_original[count]:
            print(source[count] + ' ||| ' + hyp, end='\n', file=fout)
        count += 1

330
330


In [232]:
# List with disambiguated source sentences
source = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
         
# List with nbest sentences for every source in original 
nbest_disambiguated = []
counter = 0
temp = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated.append(temp)
            counter = 0
            temp = []

print(len(source))
print(len(nbest_disambiguated))           
        
count = 0
with open('disambiguated_100_source-target_en-de.txt', 'w') as fout:
    while count < 330:
        for hyp in nbest_disambiguated[count]:
            print(source[count] + ' ||| ' + hyp, end='\n', file=fout)
        count += 1

330
330


## fast_align

In [233]:
!$FAST_ALIGN -i original_100_source-target_en-de.txt -d -o -v > original_100_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_100_source-target_en-de.txt -d -o -v > disambiguated_100_source-target_en-de_fast-aligned.txt

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................
expected target length = source length * 1.02135
ITERATION 1
.................................
  log_e likelihood: -5.74575e+06
  log_2 likelihood: -8.28937e+06
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.18938
       size counts: 95
ITERATION 2
.................................
  log_e likelihood: -946236
  log_2 likelihood: -1.36513e+06
     cross entropy: 4.92363
        perplexity: 30.3501
      posterior p0: 0.0237896
 posterior al-feat: -0.153666
       size counts: 95
  1  model al-feat: -0.164251 (tension=4)
  2  model al-feat: -0.158913 (tension=4.2117)
  3  model al-feat: -0.156355 (tension=4.31663)
  4  model al-feat: -0.155067 (tension=4.37042)
  5  model al-feat: -0.154402 (tension=4.39843)
  6  model al-feat: -0.154054 (tension=4.41314)
  7  model al-feat: -0.153871 (tension=4.4209)
  8  model al-feat: -0.153774 (tension=4.42499)
     final ten

In [234]:
# Extract target translated words to source words in original

import re

# List with original translations
translations_original = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        translations_original.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('original_100_source-target_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_original:
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    
#print(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)

330
9.554545454545455


In [235]:
# Extract target translated words to source words in disambiguated

import re

# List with original translations
translations_disambiguated = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        translations_disambiguated.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('disambiguated_100_source-target_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; !!! add 1 because of gender word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_disambiguated:
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    
#print(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)

330
10.157575757575758


## awesome_align

In [236]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_100_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_100_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

Loading the dataset...
Extracting: 33000it [00:44, 739.41it/s]


In [237]:
!awesome-align \
    --output_file "disambiguated_100_source-target_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_100_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

Loading the dataset...
Extracting: 33000it [00:41, 786.27it/s]


In [238]:
# Extract target translated words to source words in original

import re

# List with original translations
translations_original = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        translations_original.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('original_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_original:
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    
#print(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)

330
9.648484848484848


In [239]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0                
with open('unique-words_translations_original_100.txt', 'w') as fout:
    while count < 330:
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
        count += 1

In [240]:
# Extract target translated words to source words in disambiguated

import re

# List with original translations
translations_disambiguated = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        translations_disambiguated.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('disambiguated_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; !!! add 1 because of gender word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_disambiguated:
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    
#print(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)

330
10.6


In [241]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0                
with open('unique-words_translations_disambiguated_100.txt', 'w') as fout:
    while count < 330:
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
        count += 1

# Word alignement (translation-backtranslation)

## fast_align

- Extract the position of the translated ambiguous word from each sentence

In [242]:
import re
             
lineNumber = 0
counter = 0
positions_ambiguous_words_original = [] # a list of lists of indices of translated words for each ambiguous word
with open('original_100_source-target_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            positions_ambiguous_words_original.append([int(index) for index in re.findall(regex, line)])
        else:
            positions_ambiguous_words_original.append([999])
        lineNumber += 1
        
print(len(positions_ambiguous_words_original))
#print(positions_ambiguous_words_original)

33000


In [243]:
import re
             
lineNumber = 0
counter = 0
positions_ambiguous_words_disambiguated = [] # a list of lists of indices of translated words for each ambiguous word
with open('disambiguated_100_source-target_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; !!! add 1 because of gender word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            positions_ambiguous_words_disambiguated.append([int(index) for index in re.findall(regex, line)])
        else:
            positions_ambiguous_words_disambiguated.append([999])
        lineNumber += 1
        
print(len(positions_ambiguous_words_disambiguated))
#print(positions_ambiguous_words_disambiguated)

33000


- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [244]:
# List with original translated sentences
translations = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        translations.append(line.strip())
         
# List with nbest sentences for every translation in original 
nbest_original = []
counter = 0
temp = []
with open('hyp_original_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original.append(temp)
            counter = 0
            temp = []

print(len(translations))
print(len(nbest_original))          
        
count = 0
with open('original_100_translation-back_en-de.txt', 'w') as fout:
    while count < 33000:
        for hyp in nbest_original[count]:
            print(translations[count] + ' ||| ' + hyp, end='\n', file=fout)
        count += 1

33000
33000


In [245]:
# List with disambiguated translated sentences
translations = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        translations.append(line.strip())
         
# List with nbest sentences for every translation in original 
nbest_disambiguated = []
counter = 0
temp = []
with open('hyp_disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated.append(temp)
            counter = 0
            temp = []

print(len(translations))
print(len(nbest_disambiguated))           
        
count = 0
with open('disambiguated_100_translation-back_en-de.txt', 'w') as fout:
    while count < 33000:
        for hyp in nbest_disambiguated[count]:
            print(translations[count] + ' ||| ' + hyp, end='\n', file=fout)
        count += 1

33000
33000


- Word alignement

In [246]:
!$FAST_ALIGN -i original_100_translation-back_en-de.txt -d -o -v > original_100_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_100_translation-back_en-de.txt -d -o -v > disambiguated_100_translation-back_en-de_fast-aligned.txt

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
.................................................. [200000]
.................................................. [250000]
.................................................. [300000]
.................................................. [350000]
.................................................. [400000]
.................................................. [450000]
.................................................. [500000]
.................................................. [550000]
.................................................. [600000]
.................................................. [650000]
.................................................. [700000]
.................................................. [750000]
.................................................. [800000]
...

- Extract target backtranslated words

In [247]:
import re

# List with original backtranslations
backtranslations_original = []
with open('hyp_original_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations_original.append(line.strip())
             
lineNumber = 0
counter = 0
indices = []
with open('original_100_translation-back_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        positions = positions_ambiguous_words_original[counter] # exact positions of ambiguous words
        list_indices = []
        for position in positions:
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                list_indices.extend([int(index) for index in re.findall(regex, line)])
            else:
                list_indices.extend([999])
        indices.append(list_indices)
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
backtranslated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations_original:
    tokens = backtranslation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            backtranslated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()


    
#print(backtranslations_ambiguous_words)
print(len(backtranslations_ambiguous_words))

# Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
backtranslations_ambiguous_words_reduced = []
backtranslated_ambiguous_words = set() # set forces uniqueness
counter = 0
for set_words in backtranslations_ambiguous_words:
    backtranslated_ambiguous_words.update(set_words)
    counter += 1
    if (counter % 100 == 0):
        backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
        backtranslated_ambiguous_words = set()
    
print(len(backtranslations_ambiguous_words_reduced))   
    
unique_backtranslations = 0
for set_words in backtranslations_ambiguous_words_reduced:
    unique_backtranslations += len(set_words)
print(unique_backtranslations/330)

33000
330
92.68484848484849


In [248]:
import re

# List with disambiguated backtranslations
backtranslations_disambiguated = []
with open('hyp_disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations_disambiguated.append(line.strip())
             
lineNumber = 0
counter = 0
indices = []
with open('disambiguated_100_translation-back_en-de_fast-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        positions = positions_ambiguous_words_disambiguated[counter] # exact positions of ambiguous words
        list_indices = []
        for position in positions:
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                list_indices.extend([int(index) for index in re.findall(regex, line)])
            else:
                list_indices.extend([999])
        indices.append(list_indices)
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
backtranslated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations_disambiguated:
    tokens = backtranslation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            backtranslated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()
    
#print(backtranslations_ambiguous_words)
print(len(backtranslations_ambiguous_words))

# Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
backtranslations_ambiguous_words_reduced = []
backtranslated_ambiguous_words = set() # set forces uniqueness
counter = 0
for set_words in backtranslations_ambiguous_words:
    backtranslated_ambiguous_words.update(set_words)
    counter += 1
    if (counter % 100 == 0):
        backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
        backtranslated_ambiguous_words = set()

print(len(backtranslations_ambiguous_words_reduced))   
    
unique_backtranslations = 0
for set_words in backtranslations_ambiguous_words_reduced:
    unique_backtranslations += len(set_words)
print(unique_backtranslations/330)

33000
330
97.16666666666667


## awesome_align

- Extract the position of the translated ambiguous word from each sentence

In [249]:
import re
             
lineNumber = 0
counter = 0
positions_ambiguous_words_original = [] # a list of lists of indices of translated words for each ambiguous word
with open('original_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            positions_ambiguous_words_original.append([int(index) for index in re.findall(regex, line)])
        else:
            positions_ambiguous_words_original.append([999])
        lineNumber += 1
        
print(len(positions_ambiguous_words_original))
#print(positions_ambiguous_words_original)

33000


In [250]:
import re
             
lineNumber = 0
counter = 0
positions_ambiguous_words_disambiguated = [] # a list of lists of indices of translated words for each ambiguous word
with open('disambiguated_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; !!! add 1 because of gender word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            positions_ambiguous_words_disambiguated.append([int(index) for index in re.findall(regex, line)])
        else:
            positions_ambiguous_words_disambiguated.append([999])
        lineNumber += 1
        
print(len(positions_ambiguous_words_disambiguated))
#print(positions_ambiguous_words_disambiguated)

33000


- Word alignement

In [251]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_100_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_100_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

Loading the dataset...
Extracting: 3300000it [1:07:16, 817.45it/s] 


In [252]:
!awesome-align \
    --output_file "disambiguated_100_translation-back_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_100_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

Loading the dataset...
Extracting: 2704672it [57:31, 878.73it/s] 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting: 3300000it [1:09:50, 787.50it/s]


- Extract target backtranslated words

In [253]:
import re

# List with original backtranslations
backtranslations_original = []
with open('hyp_original_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations_original.append(line.strip())
             
lineNumber = 0
counter = 0
indices = []
with open('original_100_translation-back_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        positions = positions_ambiguous_words_original[counter] # exact positions of ambiguous words
        list_indices = []
        for position in positions:
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                list_indices.extend([int(index) for index in re.findall(regex, line)])
            else:
                list_indices.extend([999])
        indices.append(list_indices)
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
backtranslated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations_original:
    tokens = backtranslation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            backtranslated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()


    
#print(backtranslations_ambiguous_words)
print(len(backtranslations_ambiguous_words))

# Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
backtranslations_ambiguous_words_reduced = []
backtranslated_ambiguous_words = set() # set forces uniqueness
counter = 0
for set_words in backtranslations_ambiguous_words:
    backtranslated_ambiguous_words.update(set_words)
    counter += 1
    if (counter % 100 == 0):
        backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
        backtranslated_ambiguous_words = set()
    
print(len(backtranslations_ambiguous_words_reduced))   
    
unique_backtranslations = 0
for set_words in backtranslations_ambiguous_words_reduced:
    unique_backtranslations += len(set_words)
print(unique_backtranslations/330)

33000
330
84.96969696969697


In [254]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0                
with open('unique-words_backtranslations_original_100.txt', 'w') as fout:
    while count < 330:
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
        count += 1

In [255]:
import re

# List with disambiguated backtranslations
backtranslations_disambiguated = []
with open('hyp_disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations_disambiguated.append(line.strip())
             
lineNumber = 0
counter = 0
indices = []
with open('disambiguated_100_translation-back_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        positions = positions_ambiguous_words_disambiguated[counter] # exact positions of ambiguous words
        list_indices = []
        for position in positions:
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                list_indices.extend([int(index) for index in re.findall(regex, line)])
            else:
                list_indices.extend([999])
        indices.append(list_indices)
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
backtranslated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations_disambiguated:
    tokens = backtranslation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            backtranslated_ambiguous_words.add(tokens[ind])
    lineNumber += 1
    if (lineNumber % 100 == 0):
            backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()
    
#print(backtranslations_ambiguous_words)
print(len(backtranslations_ambiguous_words))

# Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
backtranslations_ambiguous_words_reduced = []
backtranslated_ambiguous_words = set() # set forces uniqueness
counter = 0
for set_words in backtranslations_ambiguous_words:
    backtranslated_ambiguous_words.update(set_words)
    counter += 1
    if (counter % 100 == 0):
        backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
        backtranslated_ambiguous_words = set()

print(len(backtranslations_ambiguous_words_reduced))   
    
unique_backtranslations = 0
for set_words in backtranslations_ambiguous_words_reduced:
    unique_backtranslations += len(set_words)
print(unique_backtranslations/330)

33000
330
93.89090909090909


In [256]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0                
with open('unique-words_backtranslations_disambiguated_100.txt', 'w') as fout:
    while count < 330:
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
        count += 1

# Word alignement (translation-translation)

## Tercom alignement (borrowed from Tu)
- https://github.com/TuAnh23/Perturbation-basedQE/blob/master/align_and_analyse_ambiguous_trans.py#L54-L92

### Extract target translated words to source words in original

In [127]:
# List with original source sentences; output 100 times to match backtranslation size
source = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        for i in range(10000): # append the source sentence 10000 times to match backtranslations later
            source.append(line.strip().split()) # split() tokenizes the sentence, because tercom expects tokens     

print(len(source))

# List with original backtranslations
backtranslations = []
with open('hyp_original_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations.append(line.strip().split())
        
print(len(backtranslations))

3300000
3300000


In [352]:
!git clone https://github.com/TuAnh23/Perturbation-basedQE.git

fatal: destination path 'Perturbation-basedQE' already exists and is not an empty directory.


In [128]:
%cd ./Perturbation-basedQE

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity/Perturbation-basedQE


In [129]:
import align_and_analyse_ambiguous_trans as tercom

alignments = tercom.tercom_alignment(source, backtranslations)

In [ ]:
import pandas as pd

#print(positions_ambiguous_words)

lineNumber = 0
counter = 0
indices = []
for align in alignments:
    if (lineNumber == 10000):
        lineNumber = 0
        counter += 1
    position = positions_ambiguous_words[counter] # exact position of ambiguous word
    indices.append([item[1] for item in (item for item in align if not(pd.isna(item[0]))) if item[0] == position][0])
    lineNumber += 1

print(len(indices))

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations:
    if (lineNumber == 10000):
        print("here")
        translations_ambiguous_words.append(translated_ambiguous_words)
        lineNumber = 0
        translated_ambiguous_words = set()
    backtranslation_index = backtranslations.index(backtranslation)
    if not(pd.isna(indices[backtranslation_index])):
        translated_ambiguous_words.add(backtranslation[indices[backtranslation_index]])
    lineNumber += 1
translations_ambiguous_words.append(translated_ambiguous_words) # last lines

    
#print(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)

### Extract target translated words to source words in disambiguated

In [ ]:
%cd ..

In [ ]:
# List with original source sentences; output 100 times to match backtranslation size
source = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        for i in range(10000): # append the source sentence 100 times to match backtranslations later
            source.append(line.strip().split()) # split() tokenizes the sentence, because tercom expects tokens     

print(len(source))

# List with original backtranslations
backtranslations = []
with open('hyp_disambiguated_back_100.txt', 'r') as fin:
    for line in fin:
        backtranslations.append(line.strip().split())
        
print(len(backtranslations))

In [ ]:
%cd ./Perturbation-basedQE

In [ ]:
import align_and_analyse_ambiguous_trans as tercom

alignments = tercom.tercom_alignment(source, backtranslations)

In [ ]:
import pandas as pd

#print(positions_ambiguous_words)

lineNumber = 0
counter = 0
indices = []
for align in alignments:
    if (lineNumber == 10000):
        lineNumber = 0
        counter += 1
    position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; skip gender word
    indices.append([item[1] for item in (item for item in align if not(pd.isna(item[0]))) if item[0] == position][0])
    lineNumber += 1

print(len(indices))

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for backtranslation in backtranslations:
    if (lineNumber == 10000):
        translations_ambiguous_words.append(translated_ambiguous_words)
        lineNumber = 0
        translated_ambiguous_words = set()
    backtranslation_index = backtranslations.index(backtranslation)
    if not(pd.isna(indices[backtranslation_index])):
        translated_ambiguous_words.add(backtranslation[indices[backtranslation_index]])
    lineNumber += 1
translations_ambiguous_words.append(translated_ambiguous_words) # last lines

    
#(translations_ambiguous_words)
print(len(translations_ambiguous_words))

unique_translations = 0
for set_words in translations_ambiguous_words:
    unique_translations += len(set_words)
print(unique_translations/330)


# Word occurrence

## Translation

In [ ]:
def extract_alignment_indices_translation(filename_translations):
    """
    Extract alignment indices
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_translations, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    return indices_translation

In [ ]:
indices_original = extract_alignment_indices_translation('original_source-target_en-de_awesome-aligned.txt')
indices_disambiguated = extract_alignment_indices_translation('disambiguated_source-target_en-de_awesome-aligned.txt')

In [ ]:
def extract_word_translations(filename_tokenized, filename_translations, filename_out, indices_translation):
    """
    Match alignment indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 330
    counter = 0
    for i in range(0, 330): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 330:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [ ]:
def count_word_translations(filename_tokenized, filename_translations, filename_out, indices_translation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 330
    counter = 0
    for i in range(0, 330): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 330:
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1

In [ ]:
extract_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original.txt', indices_original)
count_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original_occurrence.txt', indices_original)

extract_word_translations('tok.en_disambiguated.en', 'hyp_disambiguated.txt', 'translations_words_disambiguated.txt', indices_disambiguated)
count_word_translations('tok.en_disambiguated.en', 'hyp_disambiguated.txt', 'translations_words_disambiguated_occurrence.txt', indices_disambiguated)

## Backtranslation

In [ ]:
def extract_alignment_indices_backtranslation(filename_translations, filename_backtranslations):
    """
    Extract alignment indices
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_translations, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
       
    # Match alignement indices from translation to backtranslation
    lineNumber = 0
    counter = 0
    indices_backtranslation = []
    with open(filename_backtranslations, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            alignement_tokens = line.split()
            indices_line = []
            for index_list in indices_translation[counter]:
                index_matches = []
                for index in index_list:
                    regex = r"" + str(index) + r"-(\d)"
                    if re.findall(regex, line): 
                        index_matches.extend([int(i) for i in re.findall(regex, line)])
                    else:
                        index_matches.extend([999])
                indices_line.append(index_matches)
            indices_backtranslation.append(indices_line)
            lineNumber += 1 
    return indices_backtranslation

In [ ]:
indices_original = extract_alignment_indices_backtranslation('original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt')
indices_disambiguated = extract_alignment_indices_backtranslation('disambiguated_source-target_en-de_awesome-aligned.txt', 'disambiguated_translation-back_en-de_awesome-aligned.txt')

In [ ]:
def extract_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 330
    counter = 0
    for i in range(0, 330): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 330:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [ ]:
def count_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 330
    counter = 0
    for i in range(0, 330): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 330:
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1

In [ ]:
extract_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original.txt', indices_original)
count_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original_occurrence.txt', indices_original)

extract_word_backtranslations('tok.en_disambiguated.en', 'hyp_disambiguated_back.txt', 'backtranslations_words_disambiguated.txt', indices_disambiguated)
count_word_backtranslations('tok.en_disambiguated.en', 'hyp_disambiguated_back.txt', 'backtranslations_words_disambiguated_occurrence.txt', indices_disambiguated)

# Gender statistics

In [257]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_female


In [258]:
from enum import Enum

class GENDER(Enum):
    """
    Enumerate possible genders.
    Ignore option resolves to words that should be ignored in particular language
    """
    male = 0
    female = 1
    neutral = 2
    unknown = 3
    ignore = 4
    
DE_DETERMINERS = {"der": GENDER.male, "ein": GENDER.male, "dem": GENDER.male, #"den": GENDER.male, 
                  "einen": GENDER.male, "des": GENDER.male, "er": GENDER.male, "seiner": GENDER.male,
                  "ihn": GENDER.male, "seinen": GENDER.male, "ihm": GENDER.male, "ihren": GENDER.male,
                  "die": GENDER.female, "eine": GENDER.female, "einer": GENDER.female, "seinem": GENDER.male,
                  "ihrem": GENDER.male, "sein": GENDER.male,
                  "sie": GENDER.female, "seine": GENDER.female, "ihrer": GENDER.female, 
                  "ihr": GENDER.neutral, "ihre": GENDER.neutral, "das": GENDER.neutral,
                  "jemanden": GENDER.neutral}

def get_german_determiners(words):
    """
    Get a list of (gender)
    given a list of words.
    """
    determiners = []
    for (word_ind, word) in enumerate(words):
        word = word.lower()
        if word in DE_DETERMINERS:
            determiners.append((DE_DETERMINERS[word].name))
    return determiners

In [259]:
dets = get_german_determiners(["dem"])
print(dets)

['male']


- Extract positions of ambiguos words

In [260]:
# List with source words
words = set() # set forces uniqueness
with open('words.txt', 'r') as fin:
    for line in fin:
        words.add(line.strip())
        
ambiguous_words = set() # set forces uniqueness
positions_ambiguous_words = []

with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.add(token)
                position = tokens.index(token)
                positions_ambiguous_words.append(position)
                break
        
print(ambiguous_words)
print(len(ambiguous_words))
print(positions_ambiguous_words)
print(len(positions_ambiguous_words))

{'editor', 'assistant', 'patient', 'tailor', 'mechanic', 'firefighter', 'mover', 'laborer', 'developer', 'salesperson', 'janitor', 'cleaner', 'teenager', 'therapist', 'CEO', 'manager', 'clerk', 'chief', 'bartender', 'secretary', 'undergraduate', 'housekeeper', 'driver', 'examiner', 'specialist', 'programmer', 'dietitian', 'accountant', 'guard', 'advisor', 'broker', 'receptionist', 'sheriff', 'farmer', 'customer', 'baker', 'scientist', 'auditor', 'carpenter', 'supervisor', 'painter', 'librarian', 'writer', 'cook', 'nurse', 'attendant', 'physician', 'counselor', 'analyst', 'practitioner', 'lawyer', 'hairdresser', 'worker'}
53
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [261]:
# Extract target translated words to source words in original

import re

# List with original translations
translations_original = []
with open('hyp_original_100.txt', 'r') as fin:
    for line in fin:
        translations_original.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('original_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_original:
    if (lineNumber == 100):
            lineNumber = 0
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
    lineNumber += 1
translations_ambiguous_words.append(translated_ambiguous_words) # last lines

In [262]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_original.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0  
genders = []
male = []
female = []
with open('unique-words_translations_original_100_articles.txt', 'w') as fout:
    while count < 330:
        #print(translations_ambiguous_words[count])
        genders.append(set(get_german_determiners(translations_ambiguous_words[count])))
        male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
        female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
        count += 1

In [263]:
print(genders)
print(sum(1 for i in genders if ('male' in i and 'female' in i))) # both genders

print(male)
print(male.count(True)) # only male gender

print(female)
print(female.count(True)) # only female gender

[{'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, 

In [264]:
# Extract target translated words to source words in disambiguated

import re

# List with original translations
translations_disambiguated = []
with open('hyp_disambiguated_100.txt', 'r') as fin:
    for line in fin:
        translations_disambiguated.append(line.strip())

              
lineNumber = 0
counter = 0
indices = [] # a list of lists of indices of translated words for each ambiguous word
with open('disambiguated_100_source-target_en-de_awesome-aligned.txt', 'r') as alignments:
    for line in alignments:
        if (lineNumber == 100):
            lineNumber = 0
            counter += 1
        position = positions_ambiguous_words[counter] + 1 # exact position of ambiguous word; !!! add 1 because of gender word
        regex = r"" + str(position) + r"-(\d)"
        if re.findall(regex, line): 
            indices.append([int(index) for index in re.findall(regex, line)])
        else:
            indices.append([999])
        lineNumber += 1
        
#print(len(indices))
#print(indices)

lineNumber = 0
translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
translated_ambiguous_words = set() # set forces uniqueness
for translation in translations_disambiguated:
    if (lineNumber == 100):
            lineNumber = 0
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()
    tokens = translation.split(' ')
    if 999 not in indices[lineNumber]:
        for ind in indices[lineNumber]:
            translated_ambiguous_words.add(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
    lineNumber += 1
translations_ambiguous_words.append(translated_ambiguous_words) # last lines

In [265]:
# Add results to file

# List with original source sentences
source = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
ambiguous_words = []
with open('tok.en_disambiguated.en', 'r') as fin:
    for line in fin:
        tokens = line.split(' ')
        for token in tokens:
            if token in words:
                ambiguous_words.append(token)
                break

count = 0    
genders = []
male = []
female = []
with open('unique-words_translations_disambiguated_100_articles.txt', 'w') as fout:
    while count < 330:
        #print(translations_ambiguous_words[count])
        #print(get_german_determiners(translations_ambiguous_words[count]))
        genders.append(set(get_german_determiners(translations_ambiguous_words[count])))
        male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
        female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
        print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
        count += 1

In [266]:
print(genders)
print(sum(1 for i in genders if ('male' in i and 'female' in i))) # both genders

print(male)
print(male.count(True)) # only male gender

print(female)
print(female.count(True)) # only female gender

[{'male', 'female'}, {'male', 'female'}, {'female'}, {'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'male', 'female'}, {'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'female'}, {'female'}, {'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female', 'neutral'}, {'female'}, {'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'female', 'neutral'}, {'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'male', 'female'}, {'female'}, {'male', 'female'}, {'female'}, {'male', 'female', 'neutral'}, {'male', 'female'}, {'female'}, {'female'}, {'male', 'female', 'neutral'},